Extracting User's Google Calendar

In [23]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_ChatGPT(calendar, tasks_list) #set up the ChatGPT model
now = methods.current_time()

{'kind': 'tasks#taskLists', 'etag': '"LTgyNjg4ODUyOQ"', 'items': [{'kind': 'tasks#taskList', 'id': 'MDI1MDE5NTQzMjg1MzkzNDg1Mjk6MDow', 'etag': '"LTY0MTU3MzgzNg"', 'title': 'My Tasks', 'updated': '2024-04-13T23:19:37.627Z', 'selfLink': 'https://www.googleapis.com/tasks/v1/users/@me/lists/MDI1MDE5NTQzMjg1MzkzNDg1Mjk6MDow'}]}
My Tasks
Events for the week:
1. 4/13/2024 10:00 PM - 4/14/2024 12:00 AM: babs🕺🏽🕺🏽🕺🏽 w ketaki
2. 4/14/2024 7:00 PM - 10:00 PM: wendy's x movie night
3. 4/15/2024 10:50 AM - 11:10 AM: [EECS 583] Research Paper Presentations W24 (Marissa Bhavsar)
4. 4/15/2024 4:00 PM - 6:00 PM: 545 Project Meeting
5. 4/15/2024 7:00 PM - 9:00 PM: doms😚😋😋
6. 4/16/2024 7:00 PM - 9:00 PM: trivia @unionrec
7. 4/19/2024 6:00 PM - 8:00 PM: Cooking

Tasks for the week:
1. 598 project - Due: 4/15/2024
2. Study for 598 - Due: 4/15/2024
3. Practice presentation - Due: 4/14/2024
4. Finish EECS 583 Slides - Due: 4/15/2024

Logging Previous Conversation

In [27]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [28]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [29]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and tasks for the week, store it for your reference.{calendar}\n{tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my virtual assistant, you are an expert with managing my calendar and tasks.
You can create, move, check, list, update, delete, and handle recurring events associated with my calendar based on the user's requests.
You can also manage my tasks by prioritizing them based on deadlines and scheduling them by creating an event.
When the user doesn't provide enough information, you can ask for more details, upto 3 times. Otherwise use your best judgement.
When the user wants to schedule their tasks, use your judgement for when tasks should be scheduled based on their deadline provided. Schedule a task as an event.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Please perform the necessary actions using the Google Calendar API. Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""

# initial setup
initial_prompt = context + instruction + past_conversation

In [30]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [32]:
def chat_gpt(client, messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]

log_message("user", user_request)

In [33]:
finalized_response = False

#establishing feedback loop to chat with user
while not finalized_response:
    response = chat_gpt(client, messages)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
    else:
        finalized_response = True


I have the following tasks to schedule for the week:
1. 598 project
2. study for 598
3. practice presentation
4. Finish EECS 583 Slides

Let me find suitable time slots for these tasks throughout the week. 

Would you like me to start scheduling these tasks now?
I will start scheduling your tasks for the week based on their deadlines and your existing calendar. 

Let's begin with scheduling the tasks. 

1. **598 project:** I suggest scheduling this task on Thursday, April 18th, between 3:00 PM to 5:00 PM as it aligns with your availability.

2. **study for 598:** I recommend scheduling this task on Tuesday, April 16th, after trivia at UnionRec, around 9:00 PM for an uninterrupted study session.

3. **practice presentation:** This task can be scheduled on Thursday, April 18th, from 12:00 PM to 1:00 PM to prepare before your project meeting.

4. **Finish EECS 583 Slides:** For this task, I propose scheduling it on Monday, April 15th, after your research paper presentations, around 11:30 

In [34]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_description, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration, frequency, until)
- deleteRecurringEvent(event_id)
- updateRecurringEvent(event_id, new_description, new_duration, new_location)

Make sure to fill in the required information, such as event names, IDs, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.

Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1)'

Now, please generate the necessary function call(s) based on the user's request."""

In [35]:
messages.append({
    "role": "system",
    "content": instruction2
})
function_call = chat_gpt(client, messages)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

updateEvent("b16livsvbvqnqdlhvc35sakkdo", "Finish EECS 583 Slides", 1, "");
updateEvent("33pmlrcvmobnp4oc9t4d64ib2o_20240415T200000Z", "545 Project Meeting and Study for 598", 2, "");
updateEvent("ogmf7qgagmj778a951k8dalrgk", "Practice Presentation", 1, "");
createEvent("598 Project and Study", "2024-04-18", "15:30", 2);


In [36]:
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    parts = call.split('(')
    function_name = parts[0]
    arguments = parts[1].rstrip(')').split(', ')

    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['updateEvent("b16livsvbvqnqdlhvc35sakkdo", "Finish EECS 583 Slides", 1, "")', 'updateEvent("33pmlrcvmobnp4oc9t4d64ib2o_20240415T200000Z", "545 Project Meeting and Study for 598", 2, "")', 'updateEvent("ogmf7qgagmj778a951k8dalrgk", "Practice Presentation", 1, "")', 'createEvent("598 Project and Study", "2024-04-18", "15:30", 2)', '']
Function 'updateEvent' not found.
Function 'updateEvent' not found.
Function 'updateEvent' not found.
Create event called
The data and time is  2024-04-18 / 15:30
UTC start time:  2024-04-18 19:30:00+00:00 UTC end time:  2024-04-18 21:30:00+00:00
Event created: 598 Project and Study on 2024-04-18 at 15:30 for duration of 2 hours.


IndexError: list index out of range